This notebook is highly inspired from https://github.com/Lasagne/Recipes/blob/master/examples/lstm_text_generation.py
http://lasagne.readthedocs.io/en/latest/modules/layers/recurrent.html

In [4]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
seed = 1
lasagne.random.set_rng(np.random.RandomState(seed))

# Hyperparameters

The following are hyperparameters, except for `PRINT_FREQ`, that will have an impact on the learning algorithm.

In [ ]:
# Sequence Length
SEQ_LENGTH = 20

# Number of units in the hidden (LSTM) layers
DEPTH = 2
N_HIDDEN = 512
NON_LINEARITY = lasagne.nonlinearities.rectify

# All gradients above this will be clipped
GRAD_CLIP = 100

# Number of epochs to train the net
NUM_EPOCHS = 50

# Optimization learning rate
LEARNING_RATE = 0.01

# Batch Size
BATCH_SIZE = 128

# How often should we check the output?
PRINT_FREQ = 500

Define the optimizer to be used for the training. An optimizer can be seen as a function that takes a gradient, obtained by backpropagation, and returns an update to by applied to the current parameters. Other optimizers can be found in: [optimizer reference](http://lasagne.readthedocs.io/en/latest/modules/updates.html?highlight=update).

In [6]:
my_optimizer = lambda loss, params: lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.01, momentum=0.9)

# Dataset

The following loads the dataset (the full text Beyond Good and Evil by Friedrich Nietzsche) directly from Internet.
You can also replace the dataset by your own by using the commented line.

In [7]:
import urllib.request #For downloading the sample text file. You won't need this if you are providing your own file.
try:
    in_text = urllib.request.urlopen('https://s3.amazonaws.com/text-datasets/nietzsche.txt').read()
    #in_text = open('your_file.txt', 'r').read()
    in_text = in_text.decode("utf-8")
    print(in_text[:250])
except Exception as e:
    print("Please verify the location of the input file/URL.")
    print("A sample txt file can be downloaded from https://s3.amazonaws.com/text-datasets/nietzsche.txt")
    raise IOError('Unable to Read Text')

PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which t


The pre-processing consists in retrieving the list of symbols occuring in the text and for each symbol, convert it into an unique index. This index will be used to create an one-hot representation of the symbol that will be the input of the model.

In [8]:
chars = list(set(in_text))
data_size, vocab_size = len(in_text), len(chars)
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
print('Number of unique symbols: {}'.format(vocab_size))
print('Number of symbols in the dataset: {}'.format(data_size))

Number of unique symbols: 84
Number of symbols in the dataset: 600893


The following auxiliary function creates a minibatch in a 3D tensor (batch_size,SEQ_LENGTH,vocab_size).
For each datapoint (fixed first coordinate of the 3D matrix), there is a matrix of dimension (SEQ_LENGTH,vocab_size)
where each line contains the one-hot vector representing the character at the associated position. Notice that the sequences have all the same length (SEQ_LENGTH), which can covers many sentences. TODO: verify if it implies truncated backprop

In [9]:
def iterate_minibatch(p, batch_size = BATCH_SIZE, data=in_text, return_target=True):
    """Return a minibatch compatible with the input of the model and the associated targets
    

    Keyword arguments:
    p -- The index of the character to begin to read
    batch_size -- The number of datapoints in the current batch
    data -- The whole text
    return_target -- Create the targets (next character) associated to the sequences
    """
    x = np.zeros((batch_size,SEQ_LENGTH,vocab_size))
    y = np.zeros(batch_size)
    
    for n in range(batch_size):
        ptr = n
        for i in range(SEQ_LENGTH):
            x[n,i,char_to_ix[data[p+ptr+i]]] = 1.
        if(return_target):
            y[n] = char_to_ix[data[p+ptr+SEQ_LENGTH]]
    return x, np.array(y,dtype='int32')

# Model definition

Layers to construct recurrent networks. Recurrent layers can be used similarly to feed-forward layers except that the input shape is expected to be (batch_size, sequence_length, num_inputs). By setting the first two dimensions as None, we are allowing them to vary. They correspond to batch size and sequence length, so we will be able to feed in batches of varying size with sequences of varying length. If `only_return_final` is set, it only returns the final sequential output (e.g. for tasks where a single target value for the entire sequence is desired). In this case, Theano makes an optimization which saves memory.

In [10]:
# First, we build an input layer
# Recurrent layers expect input of shape
# (batch size, SEQ_LENGTH, num_features)

#l_in = lasagne.layers.InputLayer(shape=(None, None, vocab_size))

# We now build the LSTM layer which takes l_in as the input layer
# We clip the gradients at GRAD_CLIP to prevent the problem of exploding gradients. 

#l_forward_1 = lasagne.layers.LSTMLayer(
#    l_in, num_units=N_HIDDEN, grad_clipping=GRAD_CLIP,
#    nonlinearity=lasagne.nonlinearities.tanh, only_return_final=True)

#l_shp = lasagne.layers.ReshapeLayer(l_forward_1, (-1, N_HIDDEN))

# The output of l_forward_2 of shape (batch_size, N_HIDDEN) is then passed through the softmax nonlinearity to 
# create probability distribution of the prediction
# The output of this stage is (batch_size, vocab_size)
#l_out = lasagne.layers.DenseLayer(l_forward_2, num_units=vocab_size, W = lasagne.init.Normal(), nonlinearity=lasagne.nonlinearities.softmax)
#l_out = lasagne.layers.DenseLayer(l_shp, num_units=vocab_size, W = lasagne.init.Normal(), nonlinearity=lasagne.nonlinearities.softmax)


In [11]:
def create_lstm(
    input_var,
    nonlinearity = lasagne.nonlinearities.tanh,
    depth=2, 
    width=800):
    """
    A generic function for creating a LSTM neural network.
    
    :type input_shape: theano.tensor.var.TensorVariable
    :param learning_rate: learning rate used (factor for the stochastic
    gradient
    :type L1_reg: float
    :param L1_reg: L1-norm's weight when added to the cost (see
    regularization)
    :type L2_reg: float
    :param L2_reg: L2-norm's weight when added to the cost (see
    regularization)
    :type n_epochs: int
    :param n_epochs: maximal number of epochs to run the optimizer
    :type dataset: string
    :param dataset: the path of the MNIST dataset file from
                 http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz
   """

    # First, we build the network, starting with an input layer
    # Recurrent layers expect input of shape
    # (batch size, SEQ_LENGTH, num_features)
    network = lasagne.layers.InputLayer(shape=(None, None, vocab_size), input_var=input_var)

    # We now build the LSTM layer which takes l_in as the input layer
    # We clip the gradients at GRAD_CLIP to prevent the problem of exploding gradients. 

    for _ in range(depth-1):
    
        network = lasagne.layers.LSTMLayer(
            network, 
            num_units=N_HIDDEN, 
            grad_clipping=GRAD_CLIP,
            nonlinearity=nonlinearity)

    network = lasagne.layers.LSTMLayer(
        network, 
        num_units=N_HIDDEN, 
        grad_clipping=GRAD_CLIP,
        nonlinearity=nonlinearity,
        only_return_final=True)

    # The output of l_forward_2 of shape (batch_size, N_HIDDEN) is then passed through the softmax nonlinearity to 
    # create probability distribution of the prediction
    # The output of this stage is (batch_size, vocab_size)
    network = lasagne.layers.DenseLayer(network, num_units=vocab_size, W = lasagne.init.Normal(), nonlinearity=lasagne.nonlinearities.softmax)
    return network

# Optimization

In the following, we want to maximize the probability to output the right character given the SEQ_LENGTH previous ones. To do this, we retrieve the output of our model, which is a softmax over the alphabet or the probability of the current character, and we compare it to the actual character of the sequence. Finally, since we are using minibatches of size `BATCH_SIZE`, we compute the mean over the examples of the minibatch.

In [14]:
# Theano tensor for the targets
input_var = T.tensor3('inputs')
target_values = T.ivector('target_output')

network = create_lstm(
    input_var,
    nonlinearity = NON_LINEARITY,
    depth=DEPTH, 
    width=N_HIDDEN)
    
    
# lasagne.layers.get_output produces a variable for the output of the net
network_output = lasagne.layers.get_output(network)

# The loss function is calculated as the mean of the (categorical) cross-entropy between the prediction and target.
loss = lasagne.objectives.categorical_crossentropy(network_output,target_values).mean()

# retrieve all the parameters of the models and create the optimizer
params = lasagne.layers.get_all_params(network,trainable=True)
updates = my_optimizer(loss, params)

<class 'theano.tensor.var.TensorVariable'>


In [15]:
# Theano functions for training and computing cost
print("Compiling functions ...")
train = theano.function([input_var, target_values], loss, updates=updates, allow_input_downcast=True)
compute_loss = theano.function([input_var, target_values], loss, allow_input_downcast=True)

# In order to generate text from the network, we need the probability distribution of the next character given
# the state of the network and the input (a seed).
# In order to produce the probability distribution of the prediction, we compile a function called probs. 
    
probs = theano.function([input_var],network_output,allow_input_downcast=True)

Compiling functions ...


The next function generates text given a phrase of length at least SEQ_LENGTH.
The phrase is set using the variable generation_phrase.
The optional input "N" is used to set the number of characters of text to predict. 

In [19]:
generation_phrase = "The meaning of life is" #This phrase will be used as seed to generate text.

def predict(N=200):
    """
    Output a sequence of characters of lenght N according to the current model
    
    :type N: int
    :param N: the number of characters to generate
    """
    assert(len(generation_phrase)>=SEQ_LENGTH)
    sample_ix = []
    x,_ = iterate_minibatch(len(generation_phrase)-SEQ_LENGTH, 1, generation_phrase,0)

    for i in range(N):
        # Pick the character that got assigned the highest probability
        # ix = np.argmax(probs(x).ravel())
        
        # Alternatively, to sample from the distribution instead:
        ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:SEQ_LENGTH-1,:] = x[:,1:,:]
        x[:,SEQ_LENGTH-1,:] = 0
        x[0,SEQ_LENGTH-1,sample_ix[-1]] = 1. 

    random_snippet = generation_phrase + ''.join(ix_to_char[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

In [ ]:
print("Training ...")
print("Seed used for text generation is: " + generation_phrase)
p = 0
for it in range(int(data_size * NUM_EPOCHS / BATCH_SIZE)):
    predict() # Generate text using the p^th character as the start. 
            
    avg_cost = 0;
    for i in range(PRINT_FREQ):
        x,y = iterate_minibatch(p)

        p += SEQ_LENGTH + BATCH_SIZE - 1 
        if(p+BATCH_SIZE+SEQ_LENGTH >= data_size):
            print('Carriage Return')
            p = 0;

        avg_cost += train(x, y)
    print("Epoch {} average loss = {}".format(it*1.0*PRINT_FREQ/data_size*BATCH_SIZE, avg_cost / PRINT_FREQ))


Training ...
Seed used for text generation is: The meaning of life is
----
 The meaning of life is_2V
l:LcXb tgTzH2m1YE!wr.2x.YMyGGB:o5"PfoPQ!a!uÆ_XtWYReÆF?dNxOr4dPx-mD
HYBZl1LwnJBGc[dCCObH=2pCAKu4pqgfuäBF1kLQdPq]3Qapxq[5(CI_k:toG 9?Tz])jvHr]N
qZ5ygRë7"dFa-Q1!Ag.jMw[QQw0MRvdD3Qëe(w!_",rGAHbiæÆ9Læ: 
----
Epoch 0.0 average loss = 3.1928148078918457
----
 The meaning of life isT ldpuewidu]e;dreet niohr tdrlo
htlscEn nptts et ie Aeeuig dtwt ddomaA rontui tl mslhe tee  doltrTt chge wo thett ss  re pep ehw ry
i prgtQcfa S-O ouwgra.snthEk
 toepl tubtin eanRt,eep9sn
eicnot hioeu 
----
Epoch 0.10650814704115374 average loss = 3.1399944915771485
----
 The meaning of life is,ns iWees idiih
 tdin idttdt hadp thn rhnllt he gong e,t
IOiiaotpi y7w sirn nseaoiclaeoaro nhemto  oy s ghr ironwitfthlr yinei,
tu aic tfrs,eib
n, i, tscese hoi wd tm ?ies chyEs traty gerigsi

n sooem 
----
Epoch 0.2130162940823075 average loss = 2.9191164388656614
----
 The meaning of life isite,s
idieids teosyiso
4inny
roar,

Epoch 2.875719970111151 average loss = 1.831040940284729
----
 The meaning of life is, realif: sendem--their hull seyshivel which and commoush
exulo, af ther ask Fthes asG sums of the davied which a lear mean belande-shimality; ctosequile jurop part beong
to dit edten-ge a bukbericat  
----
Epoch 2.982228117152305 average loss = 1.8498003723621368
----
 The meaning of life is stilly be dose of the ermand are not
beants, the is not men and oun
vigio,, and that which the to morald to the self--own the hoparty pofe, ther alvestrom ham to be marcelfed,
the inadidity aponcimun 
----
Epoch 3.0887362641934586 average loss = 1.8309931552410126
----
 The meaning of life is, chunstining pactival artardzing to deariny in resine and of this pitund beconder. Shat the to, proun purttmling themsal
of jnce, pufared to its very fooes, and of "mantion agrH, corturien an sumint  
----
Epoch 3.1952444112346123 average loss = 1.7178867540359497
----
 The meaning of life is als. Their
reador to but
very dus

Epoch 5.857948087263456 average loss = 1.5085350351333617
----
 The meaning of life is beed that this perpose thee have conceiled beater, in adlier and may bevelom very propables.--Sentach _sicile: distruble thoughts appreession. On this possible bubdened authorous is
anverthelest) be  
----
Epoch 5.96445623430461 average loss = 1.4800252826213838
----
 The meaning of life is now all. The
shilt it impo philosophy- of lew as each of a more a
nature in the most dacre äo
were, which he ress. The falt; it.
Rave thing whereled, through than comrable her. Guther his but vieted  
----
Carriage Return
Epoch 6.070964381345764 average loss = 1.5405123372077942
----
 The meaning of life is the seements, and
by sarselowinged him,, perhaps
would laster of the
delightlay means and that its seluse tee manter,
against it mantice
live. Lat a necession lifesses and a smenlament!= of vainer is 
----
Epoch 6.177472528386917 average loss = 1.5573394742012023
----
 The meaning of life is even, Puupiration, 

Epoch 8.84017620441576 average loss = 1.4399874539375306
----
 The meaning of life is maturity, "bound, that the coom of
commanding, his but action, indicidation words
the people of the under
the Reduch that thereover themignation and actions anwampity,
bldunly an idsemution, which th 
----
Epoch 8.946684351456915 average loss = 1.4153251763582229
----
 The meaning of life is useful: or arrind from that here waked by
virtues, it is unfretice puls:e: than and net brengment, mis. Are to lifw
"confor, itself not seeses are not once
momeding" itenismonicalisms of his prefiaro 
----
Epoch 9.053192498498069 average loss = 1.4744211099147797
----
 The meaning of life is also religion around.

164. The for ARE, a solt, awes? Thereing) and
in the termals arrivents rools, and without absold in!upants, and, stalling"s in spirit. The furghers, is
Frolosive
and like a som 
----
Epoch 9.159700645539223 average loss = 1.439388390302658
----
 The meaning of life is almost this
eethul auts of such
jugn

Epoch 11.822404321568067 average loss = 1.3589539430141448
----
 The meaning of life is from
becomes soight in the nerman rensive, stade place
in means are allegorary in reaging, the
functs us we religions of-Alates of years, and
in, fundations and a praise in sife, the
but and mind? th 
----
Epoch 11.92891246860922 average loss = 1.2664683816432953
----
 The meaning of life is hirdelogable
with diad
yef for he would at passis of russ.

452 metafilly and erroveited, themselves, was himself the theil human masterfoute the way, the subject much more fours. Ho. A qealouds pain 
----
Epoch 12.035420615650374 average loss = 1.2769522737264634
----
 The meaning of life is among the madk of the pomitions of otherwise with his judgment freever than"shmen the laws of consequently religiouan or him. And he is growing acts, for historidunking one but an ancoicilarly expres 
----
Carriage Return
Epoch 12.141928762691528 average loss = 1.2850510275363922
----
 The meaning of life is somition of
"gr

...

Epoch 116.20038842189874 average loss = 0.05386526537872851
----
 The meaning of life is a radical
position, he once drugh
which had hither
not at once shand how ma youch over other minding, which has hitherto jide he is full of proportion and precisely what should the most picuss in Eur 
----
Epoch 116.30689656893989 average loss = 0.05811159758083522
----
 The meaning of life is fortunately to which he now, pathy for the
ord. love the excite as
henceforth a seperd is prejudices twe attrictent is impose
ay such a point of vistome but attempt,
Europe, the end, is were as in gr 
----
Epoch 116.41340471598105 average loss = 0.05587897539883852
----
 The meaning of life is fortified and instincts!i.. There is service know how to learn when the greatest proud of the
commonplace and station to contradistic morality that even the legout pow a distinct? To wrote them as a
 
----
Epoch 116.5199128630222 average loss = 0.05528562646172941
----
 The meaning of life is derual loves blated, the ve

Carriage Return
Epoch 119.18261653905104 average loss = 0.048073620606213806
----
 The meaning of life is ever the greatest day cause, men.


73

=Composed. Fut A CORPLUCT ON LELE? CHESLUTY and
hancority of doing speniness and liveders. The
shortness is even saint. Ne woml, in this duscence, all religion 
----
Epoch 119.2891246860922 average loss = 0.05010681818425655
----
 The meaning of life is twiciles" really that at every personal even sicklies for the value of former truths. Because retain for one evolved much firthess, and that has hitherto been gost one's neighbe himself about a fit t 
----
Epoch 119.39563283313335 average loss = 0.05030237819440663
----
 The meaning of life is far from of enlightenment--who knows supersions profess. The leads may say and thee; are seems into with
self-control ome work" under which does not kind or fas,"
one mis rmasos is not troubd they ha 
----
Epoch 119.5021409801745 average loss = 0.04818569608405233
----
 The meaning of life is during thei

Epoch 122.16484465620336 average loss = 0.04267347956821323
----
 The meaning of life is too Sargow. Enoung, to recognized by
the firstling exciting, sinking itself, which is conniciore NOchanged and capes there is becomiay and decently thereby already bad
conspiries ars pure at present
 
----
Epoch 122.27135280324451 average loss = 0.039590057147666814
----
 The meaning of life is so showl be the honordeatcen TR the constitute of Sughler'sism of life, to speak of FREDTO OT Accurbed and the increased in the more refined and tran this kind of man in become more advancer, thir ru 
----
Epoch 122.37786095028565 average loss = 0.04743617401830852
----
 The meaning of life is fortine, the subject, our melious and also a
latthe facts of the wicked clear; there is
a moralists and econs it is not known in the same as deep houded? the
observial himself, and perhaps we feel ph 
----
Epoch 122.4843690973268 average loss = 0.044414219913072886
----
 The meaning of life is involved in the
individua

Epoch 125.14707277335566 average loss = 0.0418375125490129
----
 The meaning of life is interrogation, men of the elements of them, whether
which has comes to cooped in the impulseder to his being sly ougnt when we are called their disposed in the soul
all masture of this nature. Ull is 
----
Epoch 125.25358092039681 average loss = 0.04352328111976385
----
 The meaning of life is able to case we feel jong parts whence the reflection of the absture which usceper in philosophers and the philoso his virtue.

113. Nourse than the least veternist, his circtes an organization in a  
----
Carriage Return
Epoch 125.36008906743797 average loss = 0.04247095392271876
----
 The meaning of life is first the siddlist of an
abdued honestofyce: to suffering of a brokged and even of an 'ego, and whatever else heavy
alluations are self-as that
the individual factly again poree of all, his virtues,  
----
Epoch 125.46659721447912 average loss = 0.043221409019082786
----
 The meaning of life is felt there

Epoch 128.12930089050795 average loss = 0.038911009101197123
----
 The meaning of life is thought, man takes simply belong to the refinement of the Clumpinate caust: than the most conceive up therefore for kinded is their auther only the causes, possession, and had he finds a new, and as  
----
Epoch 128.2358090375491 average loss = 0.038882166713476184
----
 The meaning of life is so precised one, no more by reforced in a fear its fancy obreshering the refute as preternor, which can be common to the
wear, where he love prevent--this is
knowarity to lone who is opporting collap 
----
Epoch 128.34231718459026 average loss = 0.03851695664878935
----
 The meaning of life is the earth still taste for ages, and of dever to the souther organization. The namely, preached--let it blend would be if one briality,

99OUT one afopressitored as he perceived, with which, for insta 
----
Epoch 128.4488253316314 average loss = 0.04150793338008225
----
 The meaning of life is thoughts and
simply "Warmi